# Interacting with the VAMPS-API
## Requirements:
* **[Python](https://www.python.org/downloads/)**  
 * required external packages: BeautifulSoup, lxml, requests
* **[Anaconda](https://www.continuum.io/downloads)** 
 * needed to run Jupyter notebook environment  
   
## This notebook:
* Logs into VAMPS account  
 * if you don't have an account, then [register here](https://vamps2.mbl.edu/users/signup).
* Produces a configuration file for a selected project 
 * Default: Eukaryote data from the Martha's Vineyard Coastal Observatory (MVCO), selecting one of several visualizations (default: piecharts)
* Displays visualization and saves time-stamped matrix and image files to local computer
* Uses geographic metadata for the first project to discover a second project at the same location  
 * Default: MVCO bacteria data
* Modifies the first configuration file to be able to use it for the second project
* Displays same visualization and saves files for the second project

## To use this notebook:
* Run jupyter enviroment locally by typing "jupyter notebook" into Terminal
* Make sure cells are cleared by clicking "Cell" -> "All Output" -> "Clear"
* Press run cell button
* Enter information if promtped   
* Do not run the next cell if the previous cell has an asterisk next to "In" ("In [*]:"), this means it is still processing  
* When the asterisk turns into any number, you can run the next cell  

<h3>Import relevant python packages; Allow both Python 2 and 3</h3>

In [ ]:
from __future__ import print_function
import os,sys
import requests
from bs4 import BeautifulSoup
import json, string, getpass

try:
    input = raw_input  
except NameError: # Python 3
    pass

<h3>Gets VAMPS username and password, then attempts login to VAMPS</h3>

In [ ]:
# get VAMPS username and password
vamps_username = input("Enter your VAMPS username: ")
vamps_password = getpass.getpass("Enter your VAMPS password: ")

if vamps_username == 'guest':
    raise ValueError('please register instead of using guest account')

# vamps:             https://vamps2.mbl.edu
# vampsdev (private) http://vampsdev.jbpc-np.mbl.edu:8124 
# localhost:         http://localhost:3000
VAMPS_HOST = 'https://vamps2.mbl.edu'

vamps_session = requests.Session()

def vamps_post(url_suffix, data={}, timeout=15):
    url = VAMPS_HOST + url_suffix
    return vamps_session.post(url, data=data, timeout=timeout)

# attempt login to VAMPS
r = vamps_post('/users/login', data={
        'username': vamps_username,
        'password': vamps_password
    })

# report login status
if r.url == VAMPS_HOST + '/users/login':
    raise ValueError('Login not successful')
elif r.url == VAMPS_HOST + '/':
    print('Login successful')
else:
    raise ValueError('unexpected server response')

<h3>Option to upload an existing config file or see list of datasets</h3>

In [ ]:
def input_yn(msg):
    msg += ' ("Y" or "N"): '
    resp = input(msg)
    return resp.lower()[:1] == 'y'

upload = input_yn("Do you want to use an already existing config file?")

# to upload config: 
if upload:
    file = input('Enter JSON Config File: ')
    with open(file) as f:        
        config = json.load(f)
    id_list = False
else:
    id_list = input_yn("Do you want to search through datasets or see all you have access to?")

<h3>If selected, search through datasets</h3>

In [ ]:
if id_list:
    search = input("Enter dataset you are looking for to get a list of matches: ")
    data = {
   'search_string': search,  # If not empty will search for projects with string in 
                             # project name, title or description (case insensitive)
   # Uncomment below line to include project information
   #'include_info':''        # if present, data will include project information
    }
    r = vamps_post('/api/find_user_projects', data)
    result = r.json()
    print(result)

<h3>If a configuration file was not uploaded, set config using MVCO eukaryote data as default project</h3>

In [ ]:
if not upload:
    # default config (if not uploaded); Emily B.'s MVCO eukaryote data
    config = {
        "api":"1",
        "source":"VAMPS-API",
        "update_data":1,
        "normalization":"none",               # none, maximum, frequency             
        "selected_distance":"morisita-horn",  # morisita-horn, jaccard, kulczynski, canberra bray-curtis
        "tax_depth":"family",                  # domain, phylum, klass, order, family, genus, species, strain
        "domains":["Eukarya"],                #["Archaea","Bacteria","Eukarya","Organelle","Unknown"] 
        "include_nas":"yes",                  # yes or no             
        "min_range":0,                        # integer 0-99
        "max_range":100,                      # integer 1-100

        # Must be a valid project - with correct permissions for the above user. 
        # Default is Emily B.'s MVCO eukaryote data
        'project':'MVCO_ciliate_timeseries2',   
        
        # Currently avalable: "dheatmap", "piecharts", "barcharts", "counts_matrix", "metadata_csv", "adiversity", "fheatmap", "dendrogram" 
        # Default is Alpha Diversity visualization
        'image':'piecharts'
        } 

<h3>Get and display dataset IDs for selected project</h3>

In [ ]:
# get project ids:
r = vamps_post('/api/get_dids_from_project', data=config)
config['ds_order'] = r.json()
print(config['ds_order'])

<h3>Create remote configuration and get timestamp (file_prefix)</h3>

In [ ]:
# Get timestamp to be used as a prefix for files:
r = vamps_post('/visuals/view_selection', data=config)
soup = BeautifulSoup(r.text, "lxml")  # html5lib  lxml html.parser

ts = soup.find(id="ts_for_bs").string
print("Timestamp/file prefix:",ts)

<h3>Save matrix file which is integral to VAMPS images</h3>

In [ ]:
def vamps_get_file(filename, local_file=None):
    if local_file is None:
        local_file = filename
    url = VAMPS_HOST + '/' + filename
    r = requests.get(url, stream=True)
    r.raise_for_status()
    with open(local_file, 'wb') as fout:
        for block in r.iter_content(1024):
           fout.write(block)
        
biom_matrix_file = ts+'_count_matrix.biom'
vamps_get_file(biom_matrix_file)

<h3>Save VAMPS visualization output file</h3>

In [ ]:
r = vamps_post('/api/create_image', data=config, timeout=30)

try:
    result = r.json()
except:
    raise ValueError(r.text())
    
local_filename = result['filename']
return_result = result['html']
print(local_filename)

vamps_get_file(local_filename)

print('Done writing local file:',local_filename)

<h3>Show visualization output for first dataset</h3>

In [ ]:
from IPython.core.display import display, HTML

def display_viz(html):
    style = "<style>.container { width:100% !important; }</style>"
    return HTML(style + html)

display_viz(return_result)

<h3>Get project Metadata</h3>

In [ ]:
data = {"project": config['project']}
r = vamps_post('/api/get_metadata_from_project', timeout=15, data=data)  
result = r.json()
print("Loaded metadata")

<h3>Format and show sample of Metadata, saving Latitude and Longitude information</h3>

In [ ]:
lat = 0
lon = 0

# get lat/lon
for ids in result:
    for mdname in result[ids]:
        if mdname == "longitude":
            lon = float(result[ids][mdname])
        elif mdname == "latitude":
            lat = float(result[ids][mdname]) 

def show_example_metadata(result, n=3):
    for dsid in list(result)[:n]:
        md = result[dsid]
        for mdname in md:
            print('{}: {}'.format(mdname, md[mdname]))
        print()

show_example_metadata(result)

### Use first project latitude/longitude metadata to search for next project name
### If MVCO Bacteria dataset is found, change config file

In [ ]:
half_size = 0.1

bounding_box = {
    'nw_lat': lat + half_size if lat > 0 else lat - half_size,
    'nw_lon': lon + half_size if lon > 0 else lon - half_size,
    'se_lat': lat - half_size if lat > 0 else lat + half_size,
    'se_lon': lon - half_size if lon > 0 else lon + half_size,
}

found = False

data = bounding_box
r = vamps_post('/api/find_projects_in_geo_area', timeout=15, data=data)  
result = r.json()

for sets in result:
    if sets == 'AFP_MVCO_Bv6':
        print("Found second project dataset using latitude/longitude data")
        # if MVCO bacteria data is found, change config project
        config['project'] = sets
        found = True
        break

### If second project wasn't found using latitude/longitude metadata, use the same configuration file and modify it as necessary

In [ ]:
# change config file project to second project if not found using lat/lon metadata
if not found:
    config['project'] = 'AFP_MVCO_Bv6'
config['domains'] = ["Bacteria"]
config["tax_depth"] = "family"

### Now use the same method as before to produce the visualization, first getting and displaying dataset IDs
### Exclude 4 datasets from second project

In [ ]:
# get project ids:
r = vamps_post('/api/get_dids_from_project', timeout=15, data=config)  
config['ds_order'] = r.json()

# exclude 4 datasets
if config['project'] == 'AFP_MVCO_Bv6':
    exclude = [336408, 336409, 336407, 336410]
    config['ds_order'] = [dsid for dsid in config['ds_order'] if dsid not in exclude]

<h3>Create remote configuration and get timestamp (file_prefix)</h3>

In [ ]:
# Get timestamp (filename prefix):
r = vamps_post('/visuals/view_selection', timeout=15, data=config)
soup = BeautifulSoup(r.text, "lxml")  # html5lib  lxml html.parser

ts = soup.find(id="ts_for_bs").string
print("Timestamp/file prefix:",ts)

<h3>Save matrix file which is integral to VAMPS images</h3>

In [ ]:
biom_matrix_file = ts+'_count_matrix.biom'

vamps_get_file(biom_matrix_file)

<h3>Save image file</h3>

In [ ]:
r = vamps_post('/api/create_image', timeout=30, data=config)

try:
    result = r.json()
except:
    raise ValueError(r.text)

local_filename = result['filename']
return_result = result['html']
print(local_filename)

vamps_get_file(local_filename)

print('Done writing local file: {}'.format(local_filename))

<h3>Show visualization output for second project</h3>

In [ ]:
display_viz(return_result)

<h3>Get project Metadata</h3>

In [ ]:
data = {"project": config['project']}
r = vamps_post('/api/get_metadata_from_project', timeout=15, data=data)  
result = r.json()
print("Loaded metadata")

<h3>Format and show sample of Metadata</h3>

In [ ]:
show_example_metadata(result)